# Some definitions

## Native coroutine

A function defined with `async def`. You can delegate a native coroutine to another using the `await` keyword. Even if there's no `await` keyword in a function, it is still a native coroutine if it has a `async def`. The `await` keyword cannot be used outside a native coroutine.

## Classic coroutine

A generator function that consumes data through calls on the `.send()` function and read that data using the `yield` keyword. They delegate to another classic coroutine using the `yield from` expression. Cannot be controlled by `await` and are not supported by `asyncio` anymore

## Generator based coroutines

A generator function decorated by `@types.coroutine`. Introduced in Python 3.5, this decorator makes a generator function into a coroutine that can be used with the `await` keyword. It has been discontinued in Python 3.11

## Async generator

A generator function defined with `async def` that has a `yield` on its body. It returns a generator object that offers a `__anext__()`, which is coroutine method to get the next item.

In [8]:
# Example: domain search using asyncio

# Search for domains with a Python keyword + the `.dev` suffix

import asyncio
import socket
from keyword import kwlist, softkwlist

MAX_KEYWORD_LEN = 4
KEYWORDS = sorted(kwlist + softkwlist)

async def probe(domain: str) -> tuple[str, bool]:
    loop = asyncio.get_running_loop()
    try:
        await loop.getaddrinfo(domain, None)
    except socket.gaierror:
        return (domain, False)
    return (domain, True)

async def main() -> None:
    names = (kw for kw in KEYWORDS if len(kw) <= MAX_KEYWORD_LEN)
    domains = (f'{name}.dev'.lower() for name in names)
    coros = [probe(domain) for domain in domains]
    for coro in asyncio.as_completed(coros):
        domain, found = await coro
        mark = '+' if found else ' '
        print(f'{mark} {domain}')

await main()

+ def.dev
+ and.dev
  is.dev
+ del.dev
+ not.dev
  elif.dev
+ try.dev
  with.dev
+ as.dev
  else.dev
  for.dev
  true.dev
+ in.dev
  _.dev
  if.dev
  pass.dev
  none.dev
  case.dev
+ from.dev
  or.dev


# Awaitable

The same way a `for` loop works with a iterable, the `await` call will work with a awaitable. Usually we will see the following awaitables:

- A native coroutine object: which is a function defined with `async def`
- A `asyncio.Task` , which we can get by passing a coroutine object to the `asyncio.create_task()` function.

However, not always we need to `await` for a `Task`. We can create it using the `asyncio.create_task()` and it will be scheduled to run.

Warning: always create a variable to hold the `Task` reference, because it might be deleted by the garbage collector otherwise, because the event loop use weak references to manage the tasks.

On the other, if we want to execute a function and wait until it's finished, we can use `await other_coro()`. (Question: where?)

Low level awaitables:
- A object that implements `__await__` and that returns a iterable. Example: `asyncio.Future`
- A object that was written in another language using the API Python/C
- A generator function that uses the `types.coroutine` (discontinued)



In [11]:
# Downloading flags with asyncio (same example of chapter 20)

# For now, without error handling (just the happy path)

from httpx import AsyncClient
POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'https://www.fluentpython.com/data/flags'
async def download_one(client: AsyncClient, cc: str):
    image = await get_flag(client, cc)
    # save_flag(image, f'{cc}.gif')
    print(cc, end=' ', flush=True)
    return cc

async def get_flag(client: AsyncClient, cc: str) -> bytes:
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
    resp = await client.get(url, timeout=6.1, follow_redirects=True)
    return resp.read()


async def download_many(cc_list: list[str]) -> int:
    # return asyncio.run(supervisor(cc_list))
    # The code above doesnt work inside the ipynb, so I've changed to
    # the code below and redefined the download many and main to
    # native coroutines as well
    return await supervisor(cc_list)

async def supervisor(cc_list: list[str]) -> int:
    async with AsyncClient() as client:
        to_do = [download_one(client, cc) for cc in sorted(cc_list)]
        res = await asyncio.gather(*to_do)

    return len(res)

from typing import Callable
async def main(downloader: Callable[[list[str]], int]) -> None:  # <13>
    await downloader(POP20_CC)

await main(download_many)

MX IN RU IR DE VN US ET JP TR NG FR BR EG ID PK PH CN BD CD 

# Async context managers

Just like we can perform operations using a context manager, which is a object that implements the `__enter__()` and `__exit__()` functions, we can use the `async with` in an object that implements the `__aenter__()` and `__aexit__()` functions. In this manner the following code

```python
from asyncpg import connection # Postgress async driver
tr = connection.transaction()
await tr.start()
try:
    await connection.execute("INSERT INTO mytable VALUES (1, 2, 3)")
except:
    await tr.rollback()
    raise
else:
    await tr.commit()
```

can be replaced with

```python
async with connection.transaction():
    await connection.execute("INSERT INTO mytable VALUES (1, 2, 3)")
```

because the `asyncpg.Transaction` will execute those methods in the initialization and termination function.

In [ ]:
# Improving the downloader with async context manager
# Using tqdm to animate the progress bar
# For now, without error handling (just the happy path)

from httpx import AsyncClient, HTTPStatusError
from enum import Enum
import tqdm
from collections import Counter
POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()

DownloadStatus = Enum('DownloadStatus', 'OK NOT_FOUND ERROR')
# low concurrency default to avoid errors from remote site,
# such as 503 - Service Temporarily Unavailable
DEFAULT_CONCUR_REQ = 5
MAX_CONCUR_REQ = 1000

BASE_URL = 'https://www.fluentpython.com/data/flags'
async def download_one(client: AsyncClient,
                       cc: str,
                       semaphore: asyncio.Semaphore,
                       verbose: bool) -> DownloadStatus:
    try:
        async with semaphore:
            image = await get_flag(client, cc)
    except HTTPStatusError:
        pass
    else:
        await asyncio.to_thread(save_flag, image, f'{cc}.gif') # Start a new thread (since Python 3.9)



    # save_flag(image, f'{cc}.gif')
    print(cc, end=' ', flush=True)
    return cc

async def get_flag(client: AsyncClient, cc: str) -> bytes:
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
    resp = await client.get(url, timeout=6.1, follow_redirects=True)
    resp.raise_for_status()
    return resp.content # What's the difference to .read()?


async def download_many(cc_list: list[str]) -> int:
    # return asyncio.run(supervisor(cc_list))
    # The code above doesnt work inside the ipynb, so I've changed to
    # the code below and redefined the download many and main to
    # native coroutines as well
    return await supervisor(cc_list)

async def supervisor(cc_list: list[str]) -> int:
    verbose = False
    counter = Counter[DownloadStatus] = Counter()
    semaphore = asyncio.Semaphore(DEFAULT_CONCUR_REQ)
    async with AsyncClient() as client:
        to_do = [download_one(client, cc, semaphore, verbose) for cc in sorted(cc_list)]
        to_do_iter = asyncio.as_completed(to_do)
        if not verbose:
            to_do_iter = tqdm.tqdm(to_do_iter,total=len(cc_list))
        for coro in to_do_iter:
            status = await coro


    return len(res)

from typing import Callable
async def main(downloader: Callable[[list[str]], int]) -> None:  # <13>
    await downloader(POP20_CC)

await main(download_many)

MX IN RU IR DE VN US ET JP TR NG FR BR EG ID PK PH CN BD CD 

# Delegating to threads

Since python 3.9, we can use `asyncio.to_thread()` function to spawn a thread with a blocking function. Before we would need to use the `loop.run_in_executor()` function, that has limitation to only positional arguments and the need to get a reference to the events `loop`. However, there's one advantage to use `loop.run_in_executor` which is that we can use a `ProcessPoolExecutor` that will run a separated process, which will avoid GIL disputes. But beware, the main process may end without proper finalization of the child process.

# Asynchronous iteration

The syntax `async for` works for objects that implements the `__aiter__` dunder method, which must be a regular method (not a coroutine) and it needs to return an asynchronous iterator.

A asynchronous iterator is a object that provided a `__anext__()` implementation, which returns a awaitable, most of the times a coroutine. For convention, an asynchronous iterator also implements the `__aiter__()` and returns a `self`

For example, the PostgreSQL driver `aiopg` can be used to iterate over cursor lines from a database

```python
async def go():
    pool = await aiopg.create_pool()
    async with pool.acquire() as conn:
        async with conn.cursor() as cur:
            await cur.execute("SELECT 1")
            ret = []
            async for row in cur:
                ret.append(row)
            assert ret == [(1, )]
```

## Using async generator functions

To avoid defining a object with the `__aiter__` and `__anext__` implementation, we can define a `async def` native coroutine that has a `yield` call on its body.

Side note: We can also use the `@asynccontextmanager` decorator from the `contextlib` on top of a async generator to create our asynchronous context manager in a similar manner that we have done to the sequential context manager.

To show the use of async generator function, we will expand the first example on this chapter



In [3]:
import asyncio
import socket
from collections.abc import Iterable, AsyncIterable
from typing import NamedTuple, Optional

class Result(NamedTuple):
    domain: str
    found: bool

OptionalLoop = Optional[asyncio.AbstractEventLoop]

async def probe(domain: str, loop: OptionalLoop = None) -> Result:
    if loop is None:
        loop = asyncio.get_running_loop()
    try:
        await loop.getaddrinfo(domain, None)
    except socket.gaierror:
        return Result(domain, False)
    return Result(domain, True)

async def multi_probe(domains: Iterable[str]) -> AsyncIterable[Result]:
    loop = asyncio.get_running_loop()
    coros = [probe(domain, loop) for domain in domains]
    for coro in asyncio.as_completed(coros):
        result = await coro
        yield result

async for result in multi_probe(['jisho.org', 'python.org', 'test.invalid']):
    print(*result, sep='\t')

python.org	True
test.invalid	False
jisho.org	True


# Asyncio alternative: curio

Take a look at the book to see the differences between `asyncio` and `curio`.

# Type hints to async objects

The type that a native coroutine returns is the type of the `return` that the function await, just like a normal function.

If we need to annotate a generic parameter that uses a coroutine we use:

```python
class typing.Coroutine(Awaitable[V_co], Generic[T_co, T_contra, V_co])
```